# Mushroom
https://archive.ics.uci.edu/ml/index.php
- 8124 종류의 버섯의 특징과 독의 유무를 정리해놓은 데이터셋
- 버섯 모양의 특징을 기반으로 독의 유무를 판단

한 줄 (row)이 버섯 한 종류   
1번째 컬럼은 독의 유무 : 독(p), 식용(e)   
2번째 컬럼은 버섯의 머리 모양 : b(벨 형태), c(원뿔), x(볼록), f(평평), k(혹), s(오목)   
4번째 컬럼은 버섯의 머리 색 : n(갈색), b(황갈색), c(연한갈색), g(회색)   

* 자세한 내용은 위의 사이트에서 확인

인공 신경망, 의사 결정나무, RandomForest, SVM, 로지스틱 회귀분석

In [21]:
options(warn = -1)
library(nnet)
library(party)
library(randomForest)
library(e1071)
library(caret)
library(dplyr)


Attaching package: ‘dplyr’

The following object is masked from ‘package:randomForest’:

    combine

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [2]:
mushroom <- read.csv("../Data/mushroom.csv", header = F)
head(mushroom)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [3]:
tail(mushroom)

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
8119,p,k,y,n,f,f,f,c,n,b,...,s,p,w,p,w,o,e,w,v,d
8120,e,k,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,b,c,l
8121,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,n,o,p,b,v,l
8122,e,f,s,n,f,n,a,c,b,n,...,s,o,o,p,o,o,p,b,c,l
8123,p,k,y,n,f,y,f,c,n,b,...,k,w,w,p,w,o,e,w,v,l
8124,e,x,s,n,f,n,a,c,b,y,...,s,o,o,p,o,o,p,o,c,l


In [4]:
str(mushroom)

'data.frame':	8124 obs. of  23 variables:
 $ V1 : Factor w/ 2 levels "e","p": 2 1 1 2 1 1 1 1 2 1 ...
 $ V2 : Factor w/ 6 levels "b","c","f","k",..: 6 6 1 6 6 6 1 1 6 1 ...
 $ V3 : Factor w/ 4 levels "f","g","s","y": 3 3 3 4 3 4 3 4 4 3 ...
 $ V4 : Factor w/ 10 levels "b","c","e","g",..: 5 10 9 9 4 10 9 9 9 10 ...
 $ V5 : Factor w/ 2 levels "f","t": 2 2 2 2 1 2 2 2 2 2 ...
 $ V6 : Factor w/ 9 levels "a","c","f","l",..: 7 1 4 7 6 1 1 4 7 1 ...
 $ V7 : Factor w/ 2 levels "a","f": 2 2 2 2 2 2 2 2 2 2 ...
 $ V8 : Factor w/ 2 levels "c","w": 1 1 1 1 2 1 1 1 1 1 ...
 $ V9 : Factor w/ 2 levels "b","n": 2 1 1 2 1 1 1 1 2 1 ...
 $ V10: Factor w/ 12 levels "b","e","g","h",..: 5 5 6 6 5 6 3 6 8 3 ...
 $ V11: Factor w/ 2 levels "e","t": 1 1 1 1 2 1 1 1 1 1 ...
 $ V12: Factor w/ 5 levels "?","b","c","e",..: 4 3 3 4 4 3 3 3 4 3 ...
 $ V13: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V14: Factor w/ 4 levels "f","k","s","y": 3 3 3 3 3 3 3 3 3 3 ...
 $ V15: Factor w/ 9 levels "b","c"

> 6:4 까지는 타겟의 비율이 달라도 데이터를 제외하지 않고 진행해도 된다

In [5]:
summary(mushroom)

 V1       V2       V3             V4       V5             V6       V7      
 e:4208   b: 452   f:2320   n      :2284   f:4748   n      :3528   a: 210  
 p:3916   c:   4   g:   4   g      :1840   t:3376   f      :2160   f:7914  
          f:3152   s:2556   e      :1500            s      : 576           
          k: 828   y:3244   y      :1072            y      : 576           
          s:  32            w      :1040            a      : 400           
          x:3656            b      : 168            l      : 400           
                            (Other): 220            (Other): 484           
 V8       V9            V10       V11      V12      V13      V14     
 c:6812   b:5612   b      :1728   e:3516   ?:2480   f: 552   f: 600  
 w:1312   n:2512   p      :1492   t:4608   b:3776   k:2372   k:2304  
                   w      :1202            c: 556   s:5176   s:4936  
                   n      :1048            e:1120   y:  24   y: 284  
                   g      : 752           

In [6]:
mushroom[,12]

[1] e c c e e c c c e c c c c e e e e e e e c e c c c e c c e b c e c r c b e
  [38] e b c c r e e c c c c r r r c c e e c e c c r e c c c c e c r c e b c b r
  [75] c b b r e r e e e e r e c c r e e c c c e c r c c c e c r r c c r c c c c
 [112] e b c e r e r b e e c e e c e c e c c c c b c b e b c e c r c e r c e e c
 [149] c c c e c e b r c c c c c c e r c r c c r c e c b c r c c c c c e e e c r
 [186] e c e c r e b c b c e c c c b c c c e r e c c c c c c b e r c c c c e c e
 [223] r b c c c c e c e e r e c b c c r r r c c e b c b c r e r e e c c c e c c
 [260] c r e c c b e e c c e c e c c r r r c e c e c e c c r c c r e e c c c c c
 [297] c e b e e c b c c b c c r e c e c b c e b c b b c b c c r c c e c c e c c
 [334] c c b r c e r c c c b c e b c e e e r c c r e r e r c e e e r c e r b c e
 [371] r r c e c c e c c c e e c c b e r e c c c c c e r e c c e e c c e c c c c
 [408] c c r c e r r e c b e c e c c e r c b r c e e c r c c b c b c c r c c r c
 [445] c r c r c b c c c b c e c c r r c r c c e r c c c r c c b e b e e c b c c
 [482] c b e b r c e r r b e e r r c e r c c c c c c c e e e b e e c b e e e r c
 [519] c c c c r e e c r c e r r c e r e e c e c b c e e r e c c e b c e e r b c
 [556] r e b c c c b b e c e c e e c c b c c r c e c r e c c c r r c e e e c e e
 [593] c e c e c c e e e r e r c c e r c e e e e e r c e c c e r c b r e c b e c
 [630] c e c b r r c r c e c c r r r r b c c c b c c r e e c e c r e c c e e c c
 [667] r r r c c e e b r e e r r c b c b c b c e c c c c c r e e c c e e e e e r
 [704] c r r c e c c r e c c c c c r r e r c b e r e c e c c r e e e c c b e c e
 [741] e c r c r c c c e b r c b c c c r e c r c r c e r e b e c c c c c b e r e
 [778] r c c c e c c c e e c e r c r c c e e c e e r c c e c c r c r c b c c e e
 [815] e c r r b c c e c c r c b b c b e b r r r e c e c c b e c e c e c c c b c
 [852] r e e e r b c c e r c r c c c e r e c r c c b c r c c r c b e c c r b c r
 [889] r c b r c c c r c c c c r e e e c e e e e e c e c e e c c b e e e e c b e
 [926] e e e e c e e e c e e c e e b c e e e b e e c e e e c e e e c e b b c e e
 [963] b b e e e b e e e b b e b e c b c e b c e e e e e e e c c e e e e e c b c
[1000] e b r e e r e e e e c c e r e e b r b e b e c c c e e e b c e b e e e e e
[1037] c b e e r r e b e c e e b e b b e r e c b e c e e b e e e b c e b r e e c
[1074] b r e e e e r e e c e c e c c e c e r e b e e e e e e e e e e e e e e c e
[1111] e e r e r e e e e e e e e b b e e c e e e e e e c e b c e e e b e e e e e
[1148] e e e e r e e e c e e e e c e b r e c e b e c e b e e e e c e e r r r e c
[1185] c e e e c e e c b r c e e c c c c c c e e e e e e e e e r e e e e e e e r
[1222] e e e e e b e e b e e e e e e e e e e e e e e e e e e e e b b e e e e e e
[1259] e e e e e e b e e e e b e e e e e e e e e e c e e e e e e e b e b e e e e
[1296] e b e r e e e e b e e e e e e e e e e c e e b c e b e e e e e c e e e e e
[1333] e e e b e b c e e e e b e e e e b b b e e e e e e e e e e e e e e e e e b
[1370] e e e e c e e e e e e e e e e e e e b b e b e e e e b e e e e e r b e e e
[1407] e e e e e e e e e e c e e e e e b e e e e e r e e b e e e b c e e e b e e
[1444] e e e e e b e e e e e e e e b e e b e e b e b b b e e b e e e e e e e e e
[1481] e e e e r e e e e e e e e e e e e e e e e c e e e b e b e b e e e c b e e
[1518] e c e c e e e e e b e c e e e e e e e e e e e e e e e e b e e r e e c e e
[1555] e e e c e e e e e b c b e e e e e e e e c e e e e e e b e e e e b e r e e
[1592] e e e e e b e e r b e e e e e e e b e e e e e e b e e e e e e e e b e e b
[1629] e e e e e e e e e e e e e b b e e e c e e e e e e e e e e e e e e e c c e
[1666] e e e b e c c r e e e e e e e c e c e e e e e b b r b e e e e e e e e b e
[1703] e e b e e e e b e e b e e b c c e e e e e e c e e e r e e e e e e e e b e
[1740] e e e e e r e e e e e e r e e e e e e b e e e e b e e e e b r e e e c b e
[1777] e e e e e e e e c e e e e e e e e e e e e e e e b e b b b b e b b e e b e
[1814] c e c b e e b e e e b c e b

In [7]:
for (i in 2:ncol(mushroom)) {
    print(i)
    print( cor(as.numeric(mushroom[,1]), as.numeric(mushroom[,i])) )
} 

[1] 2
[1] 0.05295056
[1] 3
[1] 0.1784461
[1] 4
[1] -0.03138409
[1] 5
[1] -0.5015304
[1] 6
[1] -0.09355164
[1] 7
[1] 0.1291999
[1] 8
[1] -0.3483868
[1] 9
[1] 0.5400244
[1] 10
[1] -0.5305662
[1] 11
[1] -0.102019
[1] 12
[1] -0.379361
[1] 13
[1] -0.3345927
[1] 14
[1] -0.2988006
[1] 15
[1] -0.1540027
[1] 16
[1] -0.1467303
[1] 17
[1] NA
[1] 18
[1] 0.1451416
[1] 19
[1] -0.2143665
[1] 20
[1] -0.4117714
[1] 21
[1] 0.171961
[1] 22
[1] 0.2986855
[1] 23
[1] 0.2171793


In [8]:
# 상관계수의 절대값이 0.5를 넘는 컬럼들(5,9,10)
mushroom2 <- mushroom[,c(1,5,9,10)]
head(mushroom2)

V1,V5,V9,V10
p,t,n,k
e,t,b,k
e,t,b,n
p,t,n,n
e,f,b,k
e,t,b,n


In [9]:
colnames(mushroom2) <- c("독유무", "멍유무", "버섯머리아래주름사이즈","버섯머리아래주름색깔")

In [10]:
samp <- createDataPartition(mushroom2$독유무, p=0.7, list=F)
length(samp)

[1] 5688

In [11]:
table(mushroom2[samp,1])


   e    p 
2946 2742 

In [12]:
for (i in 2:ncol(mushroom2)) {
    mushroom2[,i] <- as.numeric(mushroom2[,i])
}

In [13]:
data.tr <- mushroom2[samp,]
data.test <- mushroom2[-samp,]
x <- subset(data.test, select = -독유무)
y <- data.test$독유무

In [14]:
# mushroom2.norm <- as.data.frame(scale(mushroom2))
# data.tr.norm <- mushroom2.norm[samp,]
# data.test.norm <- mushroom2.norm[-samp,]
# x.norm <- subset(data.test.norm, select = -독유무)
# y.norm <- data.test.norm$독유무

In [15]:
# RandomForest
f.randomForest <- function(data.tr, data.test, x, y) {
    model.rf <- randomForest(독유무 ~ ., data=data.tr, ntree=100)
    pred <- predict(model.rf, x)
    return (mean(pred == y))
}


In [20]:
for (i in 1:5) {
    print(f.randomForest(data.tr, data.test, x, y))
}


[1] 0.8292282
[1] 0.8452381
[1] 0.8279967
[1] 0.8279967
[1] 0.8452381


In [17]:
model.rf <- randomForest(독유무 ~ ., data=data.tr, ntree=100)
pred <- predict(model.rf, x)
mean(pred == y)

[1] 0.8279967

In [18]:
table(pred, y)

    y
pred    e    p
   e 1082  239
   p  180  935